# Likelihood Analysis with Python

The python likelihood tools are a very powerful set of analysis tools that expand upon the command line tools provided with the Fermi Science Tools package. Not only can you perform all of the same likelihood analysis with the python tools that you can with the standard command line tools but you can directly access all of the model parameters.  You can more easily script a standard analysis like light curve generation.  There are also a few things built into the python tools that are not available from the command line like the calculation of upper limits.

There are many [user contributed packages](https://fermi.gsfc.nasa.gov/ssc/data/analysis/user/) built upon the python backbone of the Science Tools and we are going to highlight and use a few of them in this tutorial like likeSED, make3FGLxml, and the LATAnalysisScripts.

This sample analysis is based on the PG 1553+113 analysis performed by the LAT team and described in [Abdo, A. A. et al. 2010, ApJ, 708, 1310](http://adsabs.harvard.edu/abs/2010ApJ...708.1310A). At certain points we will refer to this article as well as the [Cicerone](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/). After you complete this tutorial you should be able to reproduce all of the data analysis performed in this publication including generating a spectrum (individual bins and a butterfly plot) and produce a light curve with the python tools.  This tutorial assumes you have the most recent [ScienceTools](https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/) installed. We will also make significant use of python, so you might want to familiarize yourself with python (there's a beginner's guide at http://wiki.python.org/moin/BeginnersGuide. This tutorial also assumes that you've gone through the non-python based [unbinned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html) thread. This tutorial should take approximately 8 hours to complete (depending on your computer's speed) if you do everything but there are some steps you can skip along the way which shave off about 4 hours of that.

> **Note:** This tutorial is generated from a [jupyter notebook](https://github.com/kialio/FSSC-Docs/blob/master/Threads/Python/python_tutorial.ipynb) which you can download and run yourself (the prefereed method). You can also run individual commands listed on this page.  If you do that, be aware that some commands must be executed in an ipython/jupyter environment.

## Get the Data

For this thread the original data were extracted from the [LAT data server](https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the paper):

* Search Center (RA,Dec) = (238.929,11.1901)
* Radius = 20 degrees
* Start Time (MET) = 239557417 seconds (2008-08-04T15:43:37)
* Stop Time (MET) = 256970880 seconds (2009-02-22T04:48:00)
* Minimum Energy = 100 MeV
* Maximum Energy = 300000 MeV

We've provided direct links to the event files as well as the spacecraft data file if you don't want to take the time to use the download server. For more information on how to download LAT data please see the [Extract LAT Data](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/explore_latdata.html) tutorial.

* [L1504241622054B65347F25_PH00.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L1504241622054B65347F25_PH00.fits)
* [L1504241622054B65347F25_PH01.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L1504241622054B65347F25_PH01.fits)
* [L1504241622054B65347F25_SC00.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L1504241622054B65347F25_SC00.fits)

Make a working directory and then download all of the files into that directory.

In [3]:
!mkdir working

In [4]:
import urllib

In [6]:
url_base = "https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/"
datafiles = ["L1504241622054B65347F25_PH00.fits", 
             "L1504241622054B65347F25_PH01.fits",
             "L1504241622054B65347F25_SC00.fits",]

In [10]:
for datafile in datafiles:
    urllib.urlretrieve(url_base+datafile,"working/"+datafile)

In [11]:
ls working/

L1504241622054B65347F25_PH00.fits  L1504241622054B65347F25_SC00.fits
L1504241622054B65347F25_PH01.fits


Now, you'll first need to make a file list with the names of your input event files:

In [13]:
ls -1 working/*PH*.fits > working/PG1553_events.list

In [15]:
mv working/L1504241622054B65347F25_SC00.fits working/PG1553_SC.fits

In the following analysis we've assumed that you've named your list of data files PG1553_events.list and the spacecraft file PG1553_SC.fits.

In [16]:
ls working/

L1504241622054B65347F25_PH00.fits  PG1553_SC.fits
L1504241622054B65347F25_PH01.fits  PG1553_events.list


## Perform Event Selections

You could follow the unbinned likelihood tutorial to perform your event selections using gtlike, gtmktime etc. directly from the command line, and then use pylikelihood later. But we're going to go ahead and use python. The gt_apps module provides methods to call these tools from within python. This'll get us used to using python.

So, let's jump into python:

Ok, we want to run gtselect inside but we first need to import the gt_apps module to gain access to it.

In [24]:
import gt_apps as my_apps

Now, you can see what objects are part of the gt_apps module by executing:

In [18]:
help(my_apps)

Help on module gt_apps:

NAME
    gt_apps - This module uses GtApp to wraps the Science Tools as python objects.

FILE
    /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/lib/python/gt_apps.py

DATA
    TsMap = <GtApp.GtApp object>
    addCubes = <GtApp.GtApp object>
    counts_map = <GtApp.GtApp object>
    diffResps = <GtApp.GtApp object>
    evtbin = <GtApp.GtApp object>
    expCube = <GtApp.GtApp object>
    expMap = <GtApp.GtApp object>
    filter = <GtApp.GtApp object>
    gtexpcube2 = <GtApp.GtApp object>
    like = <GtApp.GtApp object>
    maketime = <GtApp.GtApp object>
    model_map = <GtApp.GtApp object>
    obsSim = <GtApp.GtApp object>
    rspgen = <GtApp.GtApp object>
    srcMaps = <GtApp.GtApp object>




Which brings up the help documentation for that module (type 'x' to exit).  The python object for gtselect is called filter and we first need to set all of it's options. This is very similar to calling gtselect form the command line and inputting all of the variables interactively. It might not seem that convenient to do it this way but it's really nice once you start building up scripts (see [Building a Light Curve](#lightcurve)) and reading back options and such. For example, towards the end of this thread, we'll want to generate a light curve and we'll have to run the likelihood analysis for each datapoint. It'll be much easier to do all of this within python and change the tmin and tmax in an iterative fashion. Note that these python objects are just wrappers for the standalone tools so if you want any information on their options, see the corresponding [documentation](/ssc/data/analysis/scitools/references.html) for the standalone tool.

In [19]:
my_apps.filter['evclass'] = 128
my_apps.filter['evtype'] = 3
my_apps.filter['ra'] = 238.929
my_apps.filter['dec'] = 11.1901
my_apps.filter['rad'] = 10
my_apps.filter['emin'] = 100
my_apps.filter['emax'] = 300000
my_apps.filter['zmax'] = 90
my_apps.filter['tmin'] = 239557417
my_apps.filter['tmax'] = 256970880
my_apps.filter['infile'] = '@working/PG1553_events.list'
my_apps.filter['outfile'] = 'working/PG1553_filtered.fits'

Once this is done, run gtselect:

In [20]:
my_apps.filter.run()

time -p /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/bin/gtselect infile=@working/PG1553_events.list outfile=working/PG1553_filtered.fits ra=238.929 dec=11.1901 rad=10.0 tmin=239557417.0 tmax=256970880.0 emin=100.0 emax=300000.0 zmin=0.0 zmax=90.0 evclass=128 evclsmin=0 evclsmax=10 evtype=3 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 0.92
user 0.70
sys 0.10


Note that you can see exactly what gtselect will do if you run it by typing:

In [21]:
my_apps.filter.command()

'time -p /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/bin/gtselect infile=@working/PG1553_events.list outfile=working/PG1553_filtered.fits ra=238.929 dec=11.1901 rad=10.0 tmin=239557417.0 tmax=256970880.0 emin=100.0 emax=300000.0 zmin=0.0 zmax=90.0 evclass=128 evclsmin=0 evclsmax=10 evtype=3 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"'

You have access to any of the inputs by directly accessing the `filter['OPTIONS']` options.

Next, you need to run gtmktime. This is accessed within python via the maketime object:

In [25]:
my_apps.maketime['scfile'] = 'working/PG1553_SC.fits'
my_apps.maketime['filter'] = '(DATA_QUAL>0)&&(LAT_CONFIG==1)'
my_apps.maketime['roicut'] = 'no'
my_apps.maketime['evfile'] = 'working/PG1553_filtered.fits'
my_apps.maketime['outfile'] = 'working/PG1553_filtered_gti.fits'
my_apps.maketime.run()

time -p /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/bin/gtmktime scfile=working/PG1553_SC.fits sctable="SC_DATA" filter="(DATA_QUAL>0)&&(LAT_CONFIG==1)" roicut=no evfile=working/PG1553_filtered.fits evtable="EVENTS" outfile="working/PG1553_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 2.65
user 2.34
sys 0.17


We're using the most conservative and most commonly used cuts described in detail in the [Cicerone](http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Likelihood/Exposure.html).

## Livetime Cubes and Exposure Maps

At this point, you could make a counts map of the events we just selected using gtbin (it's called evtbin within python) and I won't discourage you but we're going to go ahead and create a livetime cube and exposure map. This might take a few minutes to complete so if you want to create a counts map and have a look at it, get these processes going and open another terminal to work on your counts map (see the [likelihood tutorial](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html#makeCountsMaps) for an example of running gtbin to produce a counts map).

### Livetime Cube

This step will take approximately 15 - 30 minutes to complete so if you want to just download the [PG1553_ltCube](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/PG1553_ltCube.fits) from us you can skip this step.

In [26]:
my_apps.expCube['evfile'] = 'working/PG1553_filtered_gti.fits'
my_apps.expCube['scfile'] = 'working/PG1553_SC.fits'
my_apps.expCube['outfile'] = 'working/PG1553_ltCube.fits'
my_apps.expCube['zmax'] = 90
my_apps.expCube['dcostheta'] = 0.025
my_apps.expCube['binsz'] = 1
my_apps.expCube.run()

time -p /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/bin/gtltcube evfile="working/PG1553_filtered_gti.fits" evtable="EVENTS" scfile=working/PG1553_SC.fits sctable="SC_DATA" outfile=working/PG1553_ltCube.fits dcostheta=0.025 binsz=1.0 phibins=0 tmin=0.0 tmax=0.0 file_version="1" zmin=0.0 zmax=90.0 chatter=2 clobber=yes debug=no gui=no mode="ql"
Working on file working/PG1553_SC.fits
.....................!
real 641.76
user 638.46
sys 1.94


While you're waiting, you might have noticed that not all of the command line science tools have an equivalent object in gt_apps.  This is easy to fix.  Say you want to use gtltcubesun from within python. Just make it a GtApp:

In [27]:
from GtApp import GtApp
expCubeSun = GtApp('gtltcubesun','Likelihood')
expCubeSun.command()

'time -p /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/bin/gtltcubesun evfile="" evtable="EVENTS" scfile= sctable="SC_DATA" outfile=expCube.fits body="SUN" dcostheta=0.025 thetasunmax=180.0 binsz=0.5 phibins=0 tmin=0.0 tmax=0.0 powerbinsun=2.7 file_version="1" zmax=180.0 chatter=2 clobber=yes debug=no gui=no mode="ql"'

### Exposure Map

In [28]:
my_apps.expMap['evfile'] = 'working/PG1553_filtered_gti.fits'
my_apps.expMap['scfile'] = 'working/PG1553_SC.fits'
my_apps.expMap['expcube'] = 'working/PG1553_ltCube.fits'
my_apps.expMap['outfile'] = 'working/PG1553_expMap.fits'
my_apps.expMap['irfs'] = 'CALDB'
my_apps.expMap['srcrad'] = 20
my_apps.expMap['nlong'] = 120
my_apps.expMap['nlat'] = 120
my_apps.expMap['nenergies'] = 37
my_apps.expMap.run()

time -p /usr/local/fermisoft/v11r5p3-fssc-20170716/x86_64-apple-darwin16.7.0/bin/gtexpmap evfile=working/PG1553_filtered_gti.fits evtable="EVENTS" scfile=working/PG1553_SC.fits sctable="SC_DATA" expcube=working/PG1553_ltCube.fits outfile=working/PG1553_expMap.fits irfs="CALDB" evtype="INDEF" srcrad=20.0 nlong=120 nlat=120 nenergies=37 submap=no nlongmin=0 nlongmax=0 nlatmin=0 nlatmax=0 chatter=2 clobber=yes debug=no gui=no mode="ql"
The exposure maps generated by this tool are meant
to be used for *unbinned* likelihood analysis only.
Do not use them for binned analyses.
Computing the ExposureMap using working/PG1553_ltCube.fits
....................!
real 337.57
user 326.21
sys 10.73


## Generate XML Model File

We need to create an XML file with all of the sources of interest within the Region of Interest (ROI) of PG 1553+113 so we can correctly model the background. For more information on the format of the model file and how to create one, see the [likelihood analysis](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html#createSourceModel) tutorial.  We'll use the user contributed tool `make3FGLxml.py` to create a model file based on the LAT 4-year LAT catalog. You'll need to download the FITS version of this file at [http://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/) and get the [make3FGLxml.py](http://fermi.gsfc.nasa.gov/ssc/data/analysis/user/make3FGLxml.py) tool from the [user contributed software](http://fermi.gsfc.nasa.gov/ssc/data/analysis/user/) page and put them both in your working directory. Also make sure you have the most recent galactic diffuse and isotropic model files which can be found at [http://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html).  In the following we assume that you have the galactic diffuse and isotropic files in your Science Tools install and we just make local symbolic links.

In [29]:
urllib.urlretrieve('http://fermi.gsfc.nasa.gov/ssc/data/analysis/user/make3FGLxml.py','make3FGLxml.py')

('make3FGLxml.py', <httplib.HTTPMessage instance at 0x10aad0710>)

In [31]:
urllib.urlretrieve('https://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/gll_psc_v16.fit',
                   'working/gll_psc_v16.fit')

('working/gll_psc_v16.fit', <httplib.HTTPMessage instance at 0x10aad0a28>)

In [46]:
!ln -s $FERMI_DIR/refdata/fermi/galdiffuse/iso_P8R2_SOURCE_V6_v06.txt working/iso_P8R2_SOURCE_V6_v06.txt

In [49]:
!ln -s $FERMI_DIR/refdata/fermi/galdiffuse/gll_iem_v06.fits working/gll_iem_v06.fits

In [53]:
ls working

L1504241622054B65347F25_PH00.fits  PG1553_filtered_gti.fits
L1504241622054B65347F25_PH01.fits  PG1553_ltCube.fits
PG1553_SC.fits                     gll_iem_v06.fits@
PG1553_events.list                 gll_psc_v16.fit
PG1553_expMap.fits                 iso_P8R2_SOURCE_V6_v06.txt@
PG1553_filtered.fits


Now that we have all of the files we need, you can generate your model file:

In [1]:
from make3FGLxml import *

This is make3FGLxml version 01r0.
The default diffuse model files and names are for pass 8 and assume you have v10r00p05 of the Fermi Science Tools or higher.


In [4]:
mymodel = srcList('working/gll_psc_v16.fit','working/PG1553_filtered_gti.fits','working/PG1553_model.xml')

In [5]:
mymodel.makeModel('working/gll_iem_v06.fit', 
                  'working/gll_iem_v06', 
                  'working/iso_P8R2_SOURCE_V6_v06.txt', 
                  'working/iso_P8R2_SOURCE_V6_v06')

Creating file and adding sources from 3FGL
Added 58 point sources and 0 extended sources


For more information on the make3FGLxml.py module, see the [usage notes(/ssc/data/analysis/user/readme_make3FGLxml.txt)

You should now have a file called 'PG1553_model.xml'. Open it up with your favorite text editor and take a look at it. It should look like [PG1553_model.xml](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/PG1553_model.xml).  There should be seven sources within 4 degrees of the ROI center, nine sources between 4 and 8 degrees, and eight sources between 8 and 12 degrees (4 of which  are beyond 10 degrees and frozen). In all, there are 38 sources beyond 10 degrees. The script designates these as outside of the ROI (which is 10 degrees) and instructs us to leave all of the variables for these sources fixed. We'll agree with this (these sources are outside of our ROI but could still affect our fit since photons from these sources could fall within our ROI due to the LAT PSF). At the bottom of the file, the two diffuse sources are listed (the galactic diffuse and extragalactic isotropic).

Notice that we've deviated a bit from the published paper here. In the paper, the LAT team only included two sources; one from the 0FGL catalog and another, non-catalog source. This is because the later LAT  catalogs had not been released at the time. However, these 3FGL sources are still in the data we've downloaded and should be modeled.

Back to looking at our XML model file, notice that all of the sources have been set up with various spectral models (see the [Cicerone](https://fermi.gsfc.nasa.gov//ssc/data/analysis/documentation/Cicerone/Cicerone_Likelihood/Model_Selection.html) for details on the different spectral models) and the module we ran filled in the values for the spectrum from the 3FGL catalog. Also notice that PG 1553+113 is listed in the model file as 3FGL J1555.7+1111 with all of the parameters filled in for us. It's actually offset from the center of our ROI by 0.008 degrees. How nice! The only problem with this is that trying to use the 3FGL model for for multiple years of data (Log Parabola) will cause us some issues for such a short time duration as we are analyzing here. Therefore we want to change the model for 3FGL J155.7+1111 to a simple power-law for the purposes of this analysis thread. You'll have to modify the relevant python scripts on your own to match whatever source model may be relevant for your data.

In the xml file, change the entry for 3FGL J1555.7+1111 from 													      

to

Save the file, return to your python window and then you're off to the races once more.....